## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);


In [2]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [3]:
data.shape

(1103665, 2)

In [4]:
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

In [5]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


Let's glance at the dataset:

In [6]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [7]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [8]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/dulan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [10]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [11]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [12]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result = [ lemmatize_stemming(word) for word in gensim.utils.simple_preprocess(text, min_len=4) ]
    return result



In [13]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [14]:
doc_sample

'rain helps dampen bushfires'

In [10]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes (it take 6 minutes on my laptop)

In [16]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [17]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0       [decid, against, communiti, broadcast, licenc]
1                       [fire, wit, must, awar, defam]
2               [call, infrastructur, protect, summit]
3                          [staff, aust, strike, rise]
4                 [strike, affect, australian, travel]
5                   [ambiti, olsson, win, tripl, jump]
6         [antic, delight, with, record, break, barca]
7    [aussi, qualifi, stosur, wast, four, memphi, m...
8          [aust, address, secur, council, over, iraq]
9                       [australia, lock, into, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [18]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [21]:
dictionary.doc2idx

<bound method Dictionary.doc2idx of <gensim.corpora.dictionary.Dictionary object at 0x125016400>>

In [22]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 against
1 broadcast
2 communiti
3 decid
4 licenc
5 awar
6 defam
7 fire
8 must
9 wit
10 call


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [23]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [24]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [25]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(80, 1), (117, 1), (491, 1), (3620, 1)]

In [26]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 80 ("bushfir") appears 1 time.
Word 117 ("help") appears 1 time.
Word 491 ("rain") appears 1 time.
Word 3620 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [27]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

# TODO
tfidf = models.TfidfModel(corpus=bow_corpus)

In [28]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tfidf[bow_corpus] 

In [29]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.31987178837257935),
 (1, 0.5646689747109036),
 (2, 0.3714475640917832),
 (3, 0.45981619161770948),
 (4, 0.47898514074940929)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [30]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = gensim.models.LdaMulticore(bow_corpus,num_topics=10, id2word=dictionary, passes=2)

In [31]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.017*"lead" + 0.015*"australia" + 0.015*"elect" + 0.014*"test" + 0.013*"market" + 0.012*"world" + 0.012*"england" + 0.011*"aussi" + 0.010*"head" + 0.010*"take" 
Words: 0


Topic: 0.054*"after" + 0.035*"fire" + 0.028*"crash" + 0.021*"polic" + 0.020*"miss" + 0.019*"continu" + 0.018*"find" + 0.014*"search" + 0.013*"die" + 0.012*"hospit" 
Words: 1


Topic: 0.056*"over" + 0.049*"polic" + 0.030*"charg" + 0.025*"court" + 0.023*"face" + 0.016*"reject" + 0.015*"drug" + 0.015*"consid" + 0.013*"accus" + 0.013*"murder" 
Words: 2


Topic: 0.027*"call" + 0.023*"plan" + 0.022*"fund" + 0.019*"govt" + 0.017*"urg" + 0.017*"council" + 0.015*"more" + 0.013*"boost" + 0.011*"work" + 0.011*"group" 
Words: 3


Topic: 0.019*"with" + 0.019*"power" + 0.017*"closer" + 0.017*"down" + 0.014*"victim" + 0.012*"play" + 0.012*"famili" + 0.011*"stand" + 0.010*"say" + 0.010*"time" 
Words: 4


Topic: 0.019*"record" + 0.019*"defend" + 0.016*"break" + 0.015*"govt" + 0.013*"sydney" + 0.012*"announc" + 0.012*"question

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [32]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf ,num_topics=10, id2word=dictionary, passes=4)

In [33]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.008*"liber" + 0.007*"whale" + 0.006*"scientist" + 0.006*"revamp" + 0.006*"black" + 0.006*"recycl" + 0.006*"rape" + 0.005*"afghan" + 0.005*"murray" + 0.004*"appoint"


Topic: 1 Word: 0.025*"closer" + 0.024*"polic" + 0.020*"crash" + 0.017*"charg" + 0.016*"over" + 0.014*"after" + 0.013*"murder" + 0.013*"investig" + 0.012*"court" + 0.011*"miss"


Topic: 2 Word: 0.009*"drink" + 0.008*"bird" + 0.007*"plead" + 0.007*"guilti" + 0.007*"grower" + 0.007*"drive" + 0.006*"over" + 0.005*"polic" + 0.005*"thousand" + 0.005*"young"


Topic: 3 Word: 0.010*"world" + 0.010*"final" + 0.008*"open" + 0.005*"win" + 0.005*"iemma" + 0.005*"ash" + 0.005*"match" + 0.005*"live" + 0.005*"socceroo" + 0.005*"draw"


Topic: 4 Word: 0.007*"tiger" + 0.007*"lead" + 0.007*"wind" + 0.006*"profit" + 0.006*"england" + 0.006*"market" + 0.006*"victori" + 0.006*"record" + 0.005*"takeov" + 0.005*"india"


Topic: 5 Word: 0.015*"govt" + 0.011*"council" + 0.011*"plan" + 0.010*"fund" + 0.009*"urg" + 0.008*"health" +

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [34]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [36]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# TODO
# Our test document is document number 4310

print(lda_model[bow_corpus[document_num]])
print("sorted")


for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

[(0, 0.020000052), (1, 0.020006496), (2, 0.25136015), (3, 0.40264922), (4, 0.020000421), (5, 0.020000014), (6, 0.020000011), (7, 0.020000022), (8, 0.02000289), (9, 0.20598073)]
sorted

Score: 0.4028201997280121	 
Topic: 0.027*"call" + 0.023*"plan" + 0.022*"fund" + 0.019*"govt" + 0.017*"urg" + 0.017*"council" + 0.015*"more" + 0.013*"boost" + 0.011*"work" + 0.011*"group"

Score: 0.2513386309146881	 
Topic: 0.056*"over" + 0.049*"polic" + 0.030*"charg" + 0.025*"court" + 0.023*"face" + 0.016*"reject" + 0.015*"drug" + 0.015*"consid" + 0.013*"accus" + 0.013*"murder"

Score: 0.20583125948905945	 
Topic: 0.018*"support" + 0.014*"south" + 0.013*"expect" + 0.013*"open" + 0.011*"centr" + 0.009*"fall" + 0.008*"about" + 0.008*"rat" + 0.008*"final" + 0.008*"cyclon"

Score: 0.02000649832189083	 
Topic: 0.054*"after" + 0.035*"fire" + 0.028*"crash" + 0.021*"polic" + 0.020*"miss" + 0.019*"continu" + 0.018*"find" + 0.014*"search" + 0.013*"die" + 0.012*"hospit"

Score: 0.02000288851559162	 
Topic: 0.041*"w

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [35]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199765086174011	 
Topic: 0.011*"rise" + 0.010*"coast" + 0.008*"price" + 0.007*"toll" + 0.007*"gold" + 0.006*"rate" + 0.006*"rat" + 0.006*"cyclon" + 0.006*"plan" + 0.006*"fire"

Score: 0.02000495418906212	 
Topic: 0.007*"tiger" + 0.007*"lead" + 0.007*"wind" + 0.006*"profit" + 0.006*"england" + 0.006*"market" + 0.006*"victori" + 0.006*"record" + 0.005*"takeov" + 0.005*"india"

Score: 0.02000459097325802	 
Topic: 0.019*"kill" + 0.012*"iraq" + 0.010*"troop" + 0.009*"bomb" + 0.009*"timor" + 0.008*"firefight" + 0.008*"hick" + 0.008*"blaze" + 0.007*"east" + 0.007*"blast"

Score: 0.020003054291009903	 
Topic: 0.015*"govt" + 0.011*"council" + 0.011*"plan" + 0.010*"fund" + 0.009*"urg" + 0.008*"health" + 0.008*"water" + 0.007*"group" + 0.007*"call" + 0.006*"over"

Score: 0.020002618432044983	 
Topic: 0.008*"burn" + 0.008*"celebr" + 0.007*"news" + 0.006*"hobart" + 0.006*"rescu" + 0.006*"boat" + 0.006*"stock" + 0.005*"snowi" + 0.005*"after" + 0.005*"fall"

Score: 0.020002596080303192	 
T

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [37]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.44664448499679565	 Topic: 0.019*"record" + 0.019*"defend" + 0.016*"break" + 0.015*"govt" + 0.013*"sydney"
Score: 0.3933541476726532	 Topic: 0.018*"support" + 0.014*"south" + 0.013*"expect" + 0.013*"open" + 0.011*"centr"
Score: 0.02000097557902336	 Topic: 0.054*"after" + 0.035*"fire" + 0.028*"crash" + 0.021*"polic" + 0.020*"miss"
Score: 0.020000342279672623	 Topic: 0.030*"death" + 0.022*"health" + 0.020*"servic" + 0.018*"rise" + 0.016*"price"
Score: 0.02000001072883606	 Topic: 0.056*"over" + 0.049*"polic" + 0.030*"charg" + 0.025*"court" + 0.023*"face"
Score: 0.020000005140900612	 Topic: 0.017*"lead" + 0.015*"australia" + 0.015*"elect" + 0.014*"test" + 0.013*"market"
Score: 0.019999999552965164	 Topic: 0.027*"call" + 0.023*"plan" + 0.022*"fund" + 0.019*"govt" + 0.017*"urg"
Score: 0.019999999552965164	 Topic: 0.019*"with" + 0.019*"power" + 0.017*"closer" + 0.017*"down" + 0.014*"victim"
Score: 0.019999999552965164	 Topic: 0.036*"kill" + 0.021*"iraq" + 0.019*"concern" + 0.018*"over

The model correctly classifies the unseen document with 'x'% probability to the X category.

In [44]:
pyLDAvis.enable_notebook()

In [52]:
from pyLDAvis import gensim
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model,bow_corpus, dictionary)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.007016 -0.240441       1        1  13.604118
2      0.231745  0.124742       2        1  11.372260
0     -0.237667  0.209416       3        1  10.227891
1      0.175289  0.105563       4        1  10.203588
7      0.089655  0.081162       5        1  10.081370
6      0.171485 -0.064545       6        1   9.441128
8     -0.045807 -0.177842       7        1   9.223075
4     -0.160786  0.103710       8        1   8.888565
9     -0.147626 -0.070849       9        1   8.772303
5     -0.083305 -0.070916      10        1   8.185693, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
262   Default   7828.000000      after   7828.000000  30.0000  30.0000
240   Default  11338.000000      polic  11338.000000  29.0000  29.0000
40    Default  14957.000000       over  14957.000000  28.0000  28.0000
61    Default   6067.000000      water   6067.000000  27.0000  27.0000
7     Default   5100.000000       fire   5100.000000  26.0000  26.0000
538   Default   5125.000000       kill   5125.000000  25.0000  25.0000
324   Default   4830.000000      charg   4830.000000  24.0000  24.0000
10    Default   5335.000000       call   5335.000000  23.0000  23.0000
145   Default   3921.000000      death   3921.000000  22.0000  22.0000
197   Default   3953.000000      crash   3953.000000  21.0000  21.0000
215   Default   4236.000000      court   4236.000000  20.0000  20.0000
102   Default   4312.000000       fund   4312.000000  19.0000  19.0000
432   Default   4108.000000       face   4108.000000  18.0000  18.0000
265   Default   2964.000000     health   2964.000000  17.0000  17.0000
103   Default   2890.000000       miss   2890.000000  16.0000  16.0000
320   Default   2636.000000     servic   2636.000000  15.0000  15.0000
371   Default   2673.000000    concern   2673.000000  14.0000  14.0000
39    Default   3550.000000       iraq   3550.000000  13.0000  13.0000
144   Default   2898.000000    continu   2898.000000  12.0000  12.0000
78    Default   2495.000000       lead   2495.000000  11.0000  11.0000
28    Default   2203.000000     record   2203.000000  10.0000  10.0000
1109  Default   2407.000000      power   2407.000000   9.0000   9.0000
342   Default   2447.000000       drug   2447.000000   8.0000   8.0000
15    Default   2690.000000       rise   2690.000000   7.0000   7.0000
2006  Default   2181.000000     closer   2181.000000   6.0000   6.0000
383   Default   2256.000000     defend   2256.000000   5.0000   5.0000
588   Default   2282.000000    support   2282.000000   4.0000   4.0000
376   Default   2596.000000     reject   2596.000000   3.0000   3.0000
304   Default   2338.000000     consid   2338.000000   2.0000   2.0000
278   Default   3930.000000       find   3930.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
1486  Topic10    216.363678       stun    217.409332   2.4980  -6.2735
1897  Topic10    187.582138     athlet    188.493118   2.4979  -6.4163
175   Topic10    501.066071      phone    503.564392   2.4978  -5.4338
1766  Topic10    726.619690      level    730.354065   2.4977  -5.0621
1981  Topic10    599.705566      board    603.847717   2.4959  -5.2541
344   Topic10    543.067871    western    546.463989   2.4965  -5.3533
2281  Topic10    312.016907  sharehold    313.699890   2.4974  -5.9074
2158  Topic10    437.019501       hall    441.515442   2.4925  -5.5705
383   Topic10   2179.432617     defend   2256.692871   2.4679  -3.9637
1242  Topic10    901.797546      right    925.177185   2.4772  -4.8461
1863  Topic10    536.713928     commit    545.573242   2.4864  -5.3650
2389  Topic10    500.418884     unveil    508.554535   2.4867  -5.4350
706   Topic10    826.752197     defenc    848.846924   2.4764  -4.9330
1855  Topic10    642.311096     hobart    6